<a href="https://colab.research.google.com/github/IlyaDenisov88/dataenj/blob/main/PySpark/UDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=4ef3d918cc0438744632ca1626921ff254463099e655676751feeee97b1c6186
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


**UDF (User-Defined Function)** в Apache Spark — это пользовательская функция, которая позволяет выполнять произвольные вычисления над данными в DataFrame.

- UDF можно использовать для применения сложной логики или вычислений, которые не поддерживаются стандартными функциями Spark SQL.
- В целом, создание UDF очень простое. Необходимо всего лишь импортнуть библиотеку udf. Рассмотрим примеры -



In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType, StringType

spark = SparkSession.builder \
    .appName("UDF Example") \
    .getOrCreate()

data = [(1, "Alice"), (2, "Bob"), (3, "Cathy"), (4, "David")]
df = spark.createDataFrame(data, ["id", "name"])

# Определяем пользовательскую функцию
def add_prefix(name):
    return "Name_" + name

# Регистрируем функцию как UDF
add_prefix_udf = udf(add_prefix, StringType())

# Применяем UDF к DataFrame
df_with_prefix = df.withColumn("prefixed_name", add_prefix_udf(col("name")))

df_with_prefix.show()

spark.stop()


+---+-----+-------------+
| id| name|prefixed_name|
+---+-----+-------------+
|  1|Alice|   Name_Alice|
|  2|  Bob|     Name_Bob|
|  3|Cathy|   Name_Cathy|
|  4|David|   Name_David|
+---+-----+-------------+



В теле функции может быть все, что угодно. Но, если Вы хотите использовать UDF в рамках чистого SQL, то конструкция создания UDF будет немного другая.

Метод `spark.udf.register` используется для регистрации пользовательских функций (UDF) в Spark, чтобы их можно было использовать в SQL-запросах. Рассмотрим пример.



In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
spark = SparkSession.builder \
    .appName("Register UDF Example") \
    .getOrCreate()


data = [(1, "Alice"), (2, "Bob"), (3, "Cathy"), (4, "David")]
df = spark.createDataFrame(data, ["id", "name"])


def name_length(name):
    return len(name)

# Регистрируем функцию как UDF с использованием spark.udf.register
spark.udf.register("name_length_udf", name_length, IntegerType())

# Создаем временную таблицу для выполнения SQL-запросов
df.createOrReplaceTempView("people")

# Используем зарегистрированную UDF в SQL-запросе
result_df = spark.sql("SELECT id, name, name_length_udf(name) as name_length FROM people")


result_df.show()


spark.stop()


+---+-----+-----------+
| id| name|name_length|
+---+-----+-----------+
|  1|Alice|          5|
|  2|  Bob|          3|
|  3|Cathy|          5|
|  4|David|          5|
+---+-----+-----------+

